In [ ]:
!pip install libfmp

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
from sys import maxsize

import libfmp.b
import libfmp.c4
import libfmp.c7
import librosa
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from random import shuffle
import random


def compute_cens_from_file(fn_wav, Fs=22050, N=4410, H=2205, ell=21, d=5):
    """Compute CENS features from file

    Notebook: C7/C7S2_AudioMatching.ipynb

    Args:
        fn_wav: Filename of wav file
        Fs: Feature rate of wav file
        N: Window size for STFT
        H: Hope size for STFT
        ell: Smoothing length
        d: Downsampling factor

    Returns:
        C_CENS: CENS features
        F_CENS: Feature rate of CENS features
        x_duration: Duration (seconds) of wav file
    """
    x, Fs = librosa.load(fn_wav, sr=Fs)
    x_duration = x.shape[0] / Fs
    X_chroma = librosa.feature.chroma_stft(y=x, sr=Fs, tuning=0, norm=None, hop_length=H, n_fft=N)
    X_CENS, Fs_CENS = libfmp.c7.compute_cens_from_chromagram(X_chroma, Fs=Fs / H, ell=ell, d=d)
    N = X_CENS.shape[1]
    return X_CENS, N, Fs_CENS, x_duration


def compute_matching_function_dtw(X, Y, step_size=2):
    """Compute CENS features from file

    Notebook: C7/C7S2_AudioMatching.ipynb

    Args:
        X: Query feature sequence (given as K x N matrix)
        Y: Database feature sequence (given as K x M matrix)
        step_size: Parameter for step size condition (1 or 2)

    Returns:
        Delta: DTW-based matching function
        C: Cost matrix
        D: Accumulated cost matrix
    """
    C = libfmp.c7.cost_matrix_dot(X, Y)
    if step_size == 1:
        D = libfmp.c7.compute_accumulated_cost_matrix_subsequence_dtw(C)
    if step_size == 2:
        D = libfmp.c7.compute_accumulated_cost_matrix_subsequence_dtw_21(C)
    N, M = C.shape
    Delta = D[-1, :] / N
    return Delta, C, D


def matches_dtw(pos, D, stepsize=2):
    """Derives matches from positions for DTW-based strategy

    Notebook: C7/C7S2_AudioMatching.ipynb

    Args:
        pos: End positions of matches
        D: Accumulated cost matrix

    Returns:
        matches: Array containing matches (start, end)
    """
    matches = np.zeros((len(pos), 2)).astype(int)
    for k in range(len(pos)):
        t = pos[k]
        matches[k, 1] = t
        if stepsize == 1:
            P = libfmp.c7.compute_optimal_warping_path_subsequence_dtw(D, m=t)
        if stepsize == 2:
            P = libfmp.c7.compute_optimal_warping_path_subsequence_dtw_21(D, m=t)
        s = P[0, 1]
        matches[k, 0] = s
    return matches


def compute_plot_matching_function_DTW(X, N, Y, ell=21, d=5, step_size=2, tau=0.2, num=5,
                                       y_lim=None):
    if y_lim is None:
        y_lim = [0, 0.35]
    color_ann = {'Theme': [0, 0, 1, 0.1], 'Match': [0, 0, 1, 0.2]}
    Delta, C, D = compute_matching_function_dtw(X, Y, step_size=step_size)
    pos = libfmp.c7.mininma_from_matching_function(Delta, rho=2 * N // 3, tau=tau, num=num)
    matches = matches_dtw(pos, D, stepsize=step_size)
    return matches

    
def main():
    wav_validation_list = [str(path.absolute()) for path in Path("/content/gdrive/MyDrive/FYP/Top_Seven_Full_Recordings_Even_Train").glob('**/*.wav')]
    wav_train_list = [str(path.absolute()) for path in Path("/content/gdrive/MyDrive/FYP/Top_Seven_Full_Recordings_Even_Train").glob('**/*.wav')]

    number_correct = 0

    training_file_cens_list = []
    for file in wav_train_list:
        print(f'Computing training cens from {Path(file).stem}')
        training_file_cens_list.append(compute_cens_from_file(file))

    for test_index, test_file in enumerate(wav_validation_list):
        file_to_match_length = {}
        print(f'Computing testing cens from {Path(test_file).stem}')
        X, N, Fs_X, x_duration = compute_cens_from_file(test_file)
        for train_index, f in enumerate(wav_train_list):
            print(f'Testing file {test_index+1} / {len(wav_validation_list)} ({Path(wav_validation_list[test_index]).stem}) - '
                  f'Comparing against {train_index+1} / {len(wav_train_list)} ({Path(wav_train_list[train_index]).stem})')
            matches = compute_plot_matching_function_DTW(X, N, training_file_cens_list[train_index][0])
            total = 0
            for (start, finish) in matches:
                total += finish - start
            file_to_match_length[Path(f).stem] = total

        file_to_match_length = {k: v for k, v in sorted(file_to_match_length.items(), key=lambda item: item[1],
                                                        reverse=True)}
        first_pair = next(iter(file_to_match_length.items()))
        path_obj = Path(test_file)
        species = str(path_obj.parents[0]).split('/')[-1]
        if species in first_pair[0]:
            number_correct += 1

    print(f'Accuracy = {(number_correct/len(wav_validation_list))*100}%')



main()


Streaming output truncated to the last 5000 lines.
Testing file 1228 / 1232 (Common_Blackbird_19) - Comparing against 1166 / 1232 (Common_Blackbird_237)
Testing file 1228 / 1232 (Common_Blackbird_19) - Comparing against 1167 / 1232 (Common_Blackbird_169)
Testing file 1228 / 1232 (Common_Blackbird_19) - Comparing against 1168 / 1232 (Common_Blackbird_271)
Testing file 1228 / 1232 (Common_Blackbird_19) - Comparing against 1169 / 1232 (Common_Blackbird_108)
Testing file 1228 / 1232 (Common_Blackbird_19) - Comparing against 1170 / 1232 (Common_Blackbird_151)
Testing file 1228 / 1232 (Common_Blackbird_19) - Comparing against 1171 / 1232 (Common_Blackbird_201)
Testing file 1228 / 1232 (Common_Blackbird_19) - Comparing against 1172 / 1232 (Common_Blackbird_135)
Testing file 1228 / 1232 (Common_Blackbird_19) - Comparing against 1173 / 1232 (Common_Blackbird_198)
Testing file 1228 / 1232 (Common_Blackbird_19) - Comparing against 1174 / 1232 (Common_Blackbird_164)
Testing file 1228 / 1232 (Commo